In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fixtures = pd.read_csv('fixtures.csv') 
player_stats = pd.read_csv('player_stats.csv')
team_defensive = pd.read_csv('team_defensive_stats.csv')
team_goalkeeping = pd.read_csv('team_goalkeeping_stats.csv')
team_passing = pd.read_csv('team_passing_stats.csv')
team_possession = pd.read_csv('team_possession_stats.csv')
team_shooting = pd.read_csv('team_shooting_stats.csv')
transfers = pd.read_csv('transfers.csv')
tables = pd.read_csv('league_tables.csv')

In [ ]:
fixtures.head()

In [ ]:
print('Fixtures:', fixtures.shape)
print('Player stats:', player_stats.shape)
print('Team defensive:', team_defensive.shape)
print('Team goalkeeping:', team_goalkeeping.shape)
print('Team passing:', team_passing.shape)
print('Team possession:', team_possession.shape)
print('Team shooting:', team_shooting.shape)
print('Transfers:', transfers.shape)
print('Tables:', tables.shape)

In [ ]:
all_teams = set(fixtures['Home']).union(fixtures['Away'])
for df in [team_shooting, team_defensive, team_goalkeeping, team_possession]:
    all_teams = all_teams.union(df['Squad'])

In [ ]:
team_id_map = {team: idx for idx, team in enumerate(sorted(all_teams), 1)}
print("Sample IDs:", dict(list(team_id_map.items())[:3]))

In [ ]:
mapping_dict = dict(list(team_id_map.items()))

Other methods failed so I am now attempting to create an id system unique to each team which will allow me to merge the data frames together

In [ ]:
#For fixtures
fixtures['home_id'] = fixtures['Home'].map(team_id_map)
fixtures['away_id'] = fixtures['Away'].map(team_id_map)

In [ ]:
#for team statistics
for df in [team_shooting, team_defensive, team_goalkeeping, team_possession]:
    df['team_id'] = df['Squad'].map(team_id_map)

In [ ]:
duplicate_cols = {'Notes', 'Attendance', 'season', 'scraped_date', 'Match Report', 'Rk'}

def clean_team_df(df):
    return df.drop(columns=[c for c in duplicate_cols if c in df.columns], errors='ignore')

team_shooting_clean = clean_team_df(team_shooting)
team_defensive_clean = clean_team_df(team_defensive)
team_goalkeeping_clean = clean_team_df(team_goalkeeping)
team_possession_clean = clean_team_df(team_possession)

team_profile = (
    team_shooting_clean.merge(
        team_defensive_clean,
        on='team_id',
        suffixes=('_shooting', '_defensive')
    ).merge(
        team_goalkeeping_clean,
        on='team_id'
    ).merge(
        team_possession_clean,
        on='team_id',
        suffixes=('', '_possession')
)
)

print("Team profile columns:", team_profile.columns.tolist())

In [ ]:
team_profile = (
    team_shooting[['team_id', 'season', 'xG', 'Sh', 'SoT']]
    .merge(team_defensive[['team_id', 'season', 'Tkl', 'Int', 'Clr']],
           on=['team_id', 'season'])
    .merge(team_goalkeeping[['team_id', 'season', 'xGA', 'GA']],
           on=['team_id', 'season'])
    .merge(team_possession[['team_id', 'season', 'Poss', 'Att 3rd']],
           on=['team_id', 'season'])
    .drop_duplicates(subset=['team_id', 'season'])
)

In [ ]:
team_profile.to_sql('team_profile', conn, index=False)

In [ ]:
import sqlite3

In [ ]:
query = """
SELECT f.*,
       home.xG as home_xG, home.Sh as home_Sh, home.Tkl as home_Tkl, home.Poss as home_Poss,
       away.xG as away_xG, away.Sh as away_Sh, away.Tkl as away_Tkl, away.Poss as away_Poss
FROM fixtures f
JOIN team_profile home 
    ON f.home_id = home.team_id 
    AND f.season = home.season
JOIN team_profile away 
    ON f.away_id = away.team_id 
    AND f.season = away.season
"""

In [ ]:
master_df = pd.read_sql(query, conn)

In [ ]:
master_df.to_csv('master dataset.csv', index=False)

In [ ]:
master_df.drop(columns=["Match Report", "scraped_date"], inplace=True)

In [ ]:
master_df.drop(columns=["Notes"], inplace=True)

In [ ]:
master_df['home_win'] = (master_df['Score'].str.split('–').str[0].astype(int) > master_df['Score'].str.split('–').str[1].astype(int)).astype(int)

Creating features for what I'll predict

In [ ]:
master_df['goal_diff'] = master_df['Score'].str.split('–').str[0].astype(int) - master_df['Score'].str.split('–').str[1].astype(int)

Some features to demonstrate rest and momentum

In [ ]:
master_df['Date'] = pd.to_datetime(master_df['Date'])
master_df = master_df.sort_values(['Home', 'Date'])

master_df['home_days_rest'] = master_df.groupby('Home')['Date'].diff().dt.days
master_df['away_days_rest'] = master_df.groupby('Away')['Date'].diff().dt.days

In [ ]:
for team_type in ['Home', 'Away']:
    master_df[f'{team_type.lower()}_form_last5'] = (
        master_df.groupby(team_type)['home_win']
        .rolling(5, min_periods=1).mean()
        .reset_index(level=0, drop=True)
    )

Features to demonstrate team strength and status

In [ ]:
for stat in ['xG', 'Poss', 'Tkl']:
    for team_type in ['home_', 'away_']:
        master_df[f'{team_type}{stat}_ema10'] = (
            master_df.groupby(team_type+'id')[f'{team_type}{stat}']
            .transform(lambda x: x.ewm(span=10).mean())
        )

In [ ]:
#To get head to head rating for teams
def calculate_h2h_avg(home_team, away_team, df, lookback_seasons=5):
    """
    Calculate average home-team goals in head-to-head matches.
    
    Args:
        home_team (str): Name of home team (e.g. 'Manchester Utd')
        away_team (str): Name of away team (e.g. 'Chelsea')
        df (DataFrame): Your master_df containing historical fixtures
        lookback_seasons (int): How many past seasons to consider
    
    Returns:
        float: Average goals scored by home team in past meetings
    """
    # Filter historical matches between these teams
    h2h_matches = df[
        ((df['Home'] == home_team) & (df['Away'] == away_team)) |
        ((df['Home'] == away_team) & (df['Away'] == home_team))
    ].sort_values('Date', ascending=False)
    
    # Optionally limit lookback period
    if 'season' in df.columns:
        current_season = df['season'].max()
        h2h_matches = h2h_matches[
            h2h_matches['season'].astype(str).str[:4].astype(int) >= 
            int(current_season[:4]) - lookback_seasons
        ]
    
    if len(h2h_matches) == 0:
        return np.nan  # No historical data
    
    # Calculate average home-team goals (adjusting for home/away)
    total_goals = 0
    for _, row in h2h_matches.iterrows():
        home_goals, away_goals = map(int, row['Score'].split('–'))
        if row['Home'] == home_team:
            total_goals += home_goals
        else:
            total_goals += away_goals
    
    return total_goals / len(h2h_matches)

In [ ]:
master_df['h2h_goal_avg'] = master_df.apply(
    lambda row: calculate_h2h_avg(row['Home'], row['Away'], master_df), 
    axis=1
)

master_df['h2h_goal_avg'] = master_df['h2h_goal_avg'].fillna(
    master_df.groupby('Home')['home_xG'].transform('mean')
)

In [ ]:
master_df.to_csv('master_df_with_h2h.csv', index=False)

In [ ]:
def h2h_stats(row, df, lookback=5):
    h2h = df[
        ((df['Home'] == row['Home']) & (df['Away'] == row['Away'])) |
        ((df['Home'] == row['Away']) & (df['Away'] == row['Home']))
    ].sort_values('Date', ascending=False).head(lookback)
    if h2h.empty:
        return np.nan, np.nan, np.nan, np.nan
    goals = []
    win_count = 0
    last_result = np.nan
    home_venue_wins = 0
    home_venue_games = 0
    away_venue_wins = 0
    away_venue_games = 0
    for i, match in h2h.iterrows():
        home_goals, away_goals = map(int, match['Score'].split('–'))
        if match['Home'] == row['Home']:
            goals.append(home_goals)
            if home_goals > away_goals:
                win_count += 1
                if i == h2h.index[0]:
                    last_result = 1
            elif home_goals == away_goals:
                if i == h2h.index[0]:
                    last_result = 0
            else:
                if i == h2h.index[0]:
                    last_result = -1
            home_venue_games += 1
            if home_goals > away_goals:
                home_venue_wins += 1
        else:
            goals.append(away_goals)
            if away_goals > home_goals:
                if i == h2h.index[0]:
                    last_result = -1
                away_venue_wins += 1
            elif away_goals == home_goals:
                if i == h2h.index[0]:
                    last_result = 0
            else:
                if i == h2h.index[0]:
                    last_result = 1
            away_venue_games += 1
    h2h_goal_avg = np.mean(goals) if goals else np.nan
    h2h_win_pct = win_count / len(h2h) if len(h2h) > 0 else np.nan
    h2h_recent_trend = last_result
    h2h_venue_impact = (home_venue_wins / home_venue_games if home_venue_games else 0) - \
                       (away_venue_wins / away_venue_games if away_venue_games else 0)
    return h2h_goal_avg, h2h_win_pct, h2h_recent_trend, h2h_venue_impact

h2h_features = master_df.apply(
    lambda row: h2h_stats(row, master_df, lookback=5), axis=1, result_type='expand'
)
master_df['h2h_goal_avg'] = h2h_features[0]
master_df['h2h_win_pct'] = h2h_features[1]
master_df['h2h_recent_trend'] = h2h_features[2]
master_df['h2h_venue_impact'] = h2h_features[3]

In [ ]:
master_df['home_danger_ratio'] = master_df['home_Sh'] / master_df['away_Tkl']
master_df['away_danger_ratio'] = master_df['away_Sh'] / master_df['home_Tkl']

In [ ]:
master_df['home_poss_eff'] = master_df['home_xG'] / master_df['home_Poss']
master_df['away_poss_eff'] = master_df['away_xG'] / master_df['away_Poss']

Lastly I want to create an Elo rating system similar to that in Chess as it tends to have high predictive features

In [ ]:
initial_elo = 1500
all_teams = set(master_df['Home']).union(set(master_df['Away']))
elo_ratings = {team: initial_elo for team in all_teams}

K_FACTOR = 30      
HOME_ADVANTAGE = 100
MARGIN_FACTOR = 1.0

elo_history_data = []

In [ ]:
master_df = master_df.sort_values('Date')

for idx, row in master_df.iterrows():
    home_team, away_team = row['Home'], row['Away']
    home_goals, away_goals = map(int, row['Score'].split('–'))
    goal_diff = abs(home_goals - away_goals)
    
    # Get current ratings
    home_elo = elo_ratings[home_team]
    away_elo = elo_ratings[away_team]
    
    # Store pre-match ratings
    master_df.at[idx, 'home_elo'] = home_elo
    master_df.at[idx, 'away_elo'] = away_elo
    
    # Calculate expected outcome
    expected_home_win = 1 / (1 + 10 ** ((away_elo - home_elo - HOME_ADVANTAGE) / 400))
    master_df.at[idx, 'elo_expected_home_win'] = expected_home_win
    
    # Update ratings based on actual outcome
    if home_goals > away_goals:  # Home win
        new_home, new_away = update_elo(home_elo, away_elo, goal_diff, HOME_ADVANTAGE)
    elif away_goals > home_goals:  # Away win
        new_away, new_home = update_elo(away_elo, home_elo, goal_diff, 0)
    else:  # Draw
        expected_draw = 1 - abs(expected_home_win - (1 - expected_home_win))
        home_change = K_FACTOR * 0.5 * (0.5 - expected_home_win)
        new_home, new_away = home_elo + home_change, away_elo - home_change
    
    # Update ratings and store history
    elo_change_home = new_home - home_elo
    elo_change_away = new_away - away_elo
    
    elo_ratings[home_team] = new_home
    elo_ratings[away_team] = new_away
    
    # Store detailed history
    elo_history_data.append({
        'date': row['Date'],
        'team': home_team,
        'elo': new_home,
        'elo_change': elo_change_home,
        'opponent': away_team,
        'is_home': True
    })
    elo_history_data.append({
        'date': row['Date'],
        'team': away_team,
        'elo': new_away,
        'elo_change': elo_change_away,
        'opponent': home_team,
        'is_home': False
    })

# Create the elo_history_df
elo_history_df = pd.DataFrame(elo_history_data)

In [ ]:
def recent_elo_form(team, current_date, matches=5):
    """Calculate average Elo change in last N matches for a team"""
    team_history = elo_history_df[
        (elo_history_df['team'] == team) & 
        (elo_history_df['date'] < current_date)
    ].sort_values('date', ascending=False).head(matches)
    
    if len(team_history) == 0:
        return 0
    return team_history['elo_change'].mean()

# Apply to dataframe
master_df['home_elo_momentum'] = master_df.apply(
    lambda x: recent_elo_form(x['Home'], x['Date'], matches=5), 
    axis=1
)
master_df['away_elo_momentum'] = master_df.apply(
    lambda x: recent_elo_form(x['Away'], x['Date'], matches=5), 
    axis=1
)

In [ ]:
# Elo difference features
master_df['elo_diff'] = master_df['home_elo'] - master_df['away_elo']
master_df['elo_win_prob'] = 1 / (1 + 10 ** ((master_df['away_elo'] - master_df['home_elo'] - HOME_ADVANTAGE) / 400))

# Strength-adjusted features
master_df['home_elo_advantage'] = master_df['home_elo'] / master_df['away_elo']
master_df['combined_elo'] = master_df['home_elo'] + master_df['away_elo']

# Recent performance features
master_df['home_elo_trend'] = master_df['home_elo_momentum'] / master_df['home_elo']
master_df['away_elo_trend'] = master_df['away_elo_momentum'] / master_df['away_elo']

In [ ]:
print("Top 5 teams by Elo:")
top_teams = sorted(elo_ratings.items(), key=lambda x: x[1], reverse=True)[:5]
for team, elo in top_teams:
    print(f"{team}: {elo:.0f}")

Here we can see the top 5 teams based on the elo system

In [ ]:
if 'home_win' in master_df.columns:
    from sklearn.metrics import roc_auc_score
    auc = roc_auc_score(master_df['home_win'], master_df['elo_win_prob'])
    print(f"Elo prediction AUC: {auc:.3f}")

We can see it has prediction score of 70.9%

In [ ]:
master_df.to_csv("master_df_final.csv", index=False)

In [ ]:
ID_TEAM_MAP = {v: k for k, v in mapping_dict.items()}

In [ ]:
features_to_drop = [
    'Score', 'home_win', 'goal_diff', 'xG', 'xG.1',
    'Home', 'Away', 'Venue', 'Referee',
    'Wk', 'Day', 'Time', 'Attendance',
    'Date', 'home_id', 'away_id', 'season'
]

In [ ]:
sns.pairplot(master_df, hue='home_win', vars=X.columns)

In [ ]:
X = master_df.drop(columns=features_to_drop, errors='ignore')
y = master_df['home_win']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

In [ ]:
param_grid = {
    'n_estimators': [10, 20, 50, 75, 100, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.3],
    'class_weight': [None, 'balanced']
}

In [ ]:
grid_cv = GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2,
    refit=True
)

In [ ]:
grid_cv.fit(X_train, y_train)

In [ ]:
best_rfc = grid_cv.best_estimator_
print("Best params:", grid_cv.best_params_)

In [ ]:
y_pred = best_rfc.predict(X_test)
y_proba = best_rfc.predict_proba(X_test)[:, 1]

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
cmd = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=best_rfc.classes_)

In [ ]:
cmd.plot()

In [ ]:
print("Test ROC AUC:", roc_auc_score(y_test, y_proba))

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', alpha=0.6, label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Home Win Prediction')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_rfc.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance.head(15))
plt.title('Top 15 Feature Importances')
plt.tight_layout()
plt.show()

print("Top 10 Most Important Features:")
print(feature_importance.head(10))

Now to create the API

In [ ]:
import joblib
import json

In [ ]:
joblib.dump(best_rfc, 'premier_league_predictor.pkl')

In [ ]:
joblib.dump(mapping_dict, 'team_mapping.pkl')

In [ ]:
with open('feature_list.json', 'w') as f:
    json.dump(list(X_train.columns), f)

In [ ]:
def predict_match_production(home_team_data, away_team_data):
    """
    Production prediction function
    home_team_data: dict with feature values for home team
    away_team_data: dict with feature values for away team
    """
    model = joblib.load('premier_league_predictor.pkl')
    feature_list = json.load(open('feature_list.json'))
    
    features = {}
    for feature in feature_list:
        if feature.startswith('home_'):
            features[feature] = home_team_data[feature.replace('home_', '')]
        elif feature.startswith('away_'):
            features[feature] = away_team_data[feature.replace('away_', '')]
        else:
            features[feature] = (home_team_data.get(feature, 0) + away_team_data.get(feature, 0)) / 2

    input_df = pd.DataFrame([features])[feature_list]
    prediction = model.predict(input_df)[0]
    probability = model.predict_proba(input_df)[0]
    
    return {
        'prediction': 'Home Win' if prediction == 1 else 'Away Win/Draw',
        'home_win_probability': float(probability[1]),
        'confidence': float(max(probability))
    }